## Imports

In [99]:
import csv

import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer

import pickle

import numpy as np
import pandas as pd

import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

from gensim import corpora, models, similarities, matutils

import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn
from sklearn.manifold import TSNE

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

%config InlineBackend.figure_formats = ['retina']

from sklearn import linear_model
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.linear_model import RidgeCV


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import SCORERS

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import lars_path


import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy #related to statsmodels

import seaborn as sns
import matplotlib.pyplot as plt

## Open Pickles

### For LSA and NMF

In [2]:
with open('count_vec_spacy.pickle','rb') as read_file:
    count_vec_spacy = pickle.load(read_file)

In [3]:
with open('tf_idf_spacy.pickle','rb') as read_file:
    tf_idf_spacy = pickle.load(read_file)

In [4]:
tf_idf_spacy.head()

,After_Clean,10 minute,20 night,30 minute,amaze view,aura restaurant,away hustle,away hustle bustle,away village,away whistler,...,whistler blackcomb,whistler creekside,whistler mountain,whistler stay,whistler time,whistler village,wife stay,wish stay,wish stay long,wonderful stay
_id,,,,,,,,,,,,,,,,,,,,,
5d5d83cac56427b284d5398f,reasonably price place stay canada amaze servi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53990,spend 3 night enjoyable impress location away ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.228178,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53991,bump property early room ready happen desk new...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53992,desk staff geoff australian look bike kayak ge...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53993,wonderful choice look away surrounding lodge s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


### For LDA

In [5]:
with open('X_cvspacy.pickle','rb') as read_file:
    X_cvspacy = pickle.load(read_file)
    

with open('id2word_cvspacy.pickle','rb') as read_file:
    id2word_cvspacy = pickle.load(read_file)

In [6]:
with open('X_tfispacy.pickle','rb') as read_file:
    X_tfispacy = pickle.load(read_file)
    

with open('id2word_tfispacy.pickle','rb') as read_file:
    id2word_tfispacy = pickle.load(read_file)

### For Regression

In [97]:
with open('y.pickle','rb') as read_file:
    y = pickle.load(read_file)

## Drop Review Text Column

In [7]:
count_vec_spacy.drop('After_Clean', inplace = True, axis = 'columns')
count_vec_spacy.head()

,10 minute,20 night,30 minute,amaze view,aura restaurant,away hustle,away hustle bustle,away village,away whistler,balcony overlook,...,whistler blackcomb,whistler creekside,whistler mountain,whistler stay,whistler time,whistler village,wife stay,wish stay,wish stay long,wonderful stay
_id,,,,,,,,,,,,,,,,,,,,,
5d5d83cac56427b284d5398f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5d5d83cac56427b284d53990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5d5d83cac56427b284d53991,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5d5d83cac56427b284d53992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5d5d83cac56427b284d53993,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
tf_idf_spacy.drop('After_Clean', inplace = True, axis = 'columns')
tf_idf_spacy.head()

,10 minute,20 night,30 minute,amaze view,aura restaurant,away hustle,away hustle bustle,away village,away whistler,balcony overlook,...,whistler blackcomb,whistler creekside,whistler mountain,whistler stay,whistler time,whistler village,wife stay,wish stay,wish stay long,wonderful stay
_id,,,,,,,,,,,,,,,,,,,,,
5d5d83cac56427b284d5398f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.228178,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## Topic modeling Variables

In [65]:
n_topics = 4
n_words = 10

## Latent Semantic Analysis

### Plotting Function

In [66]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

### Using Count Vec Spacy

In [67]:
lsa1 = TruncatedSVD(n_topics)
doc_topic1 = lsa1.fit_transform(count_vec_spacy)
lsa1.explained_variance_ratio_

array([0.0386671 , 0.0341906 , 0.02434683, 0.02106245])

In [68]:
topic_word1 = pd.DataFrame(lsa1.components_.round(3),
             columns = count_vec_spacy.columns)
topic_word1

,10 minute,20 night,30 minute,amaze view,aura restaurant,away hustle,away hustle bustle,away village,away whistler,balcony overlook,...,whistler blackcomb,whistler creekside,whistler mountain,whistler stay,whistler time,whistler village,wife stay,wish stay,wish stay long,wonderful stay
0,0.041,0.011,0.023,0.009,0.049,0.030,0.025,0.017,0.022,0.024,...,0.020,0.024,0.015,0.019,0.013,0.583,0.012,0.016,0.012,0.013
1,-0.007,0.007,-0.014,0.003,0.019,-0.000,-0.004,0.010,-0.016,-0.009,...,-0.014,-0.011,-0.011,0.005,0.003,-0.583,0.000,0.005,0.004,-0.003
2,0.000,-0.012,0.006,-0.017,-0.050,-0.001,-0.007,-0.021,0.020,-0.026,...,0.000,0.002,-0.003,-0.007,-0.002,0.423,-0.008,-0.010,-0.010,-0.011
3,0.024,0.004,0.011,-0.001,0.026,-0.003,-0.004,0.001,0.012,-0.013,...,0.023,-0.001,0.012,0.007,0.004,0.163,-0.007,-0.010,-0.009,-0.002


In [69]:
display_topics(lsa1, count_vec_spacy.columns, n_words)


Topic  0
whistler village, lake view, free shuttle, view lake, hot tub, shuttle service, view room, staff friendly, stay night, highly recommend

Topic  1
lake view, view room, lake view room, hot tub, staff friendly, view suite, lake view suite, room lake, room lake view, suite lake

Topic  2
whistler village, lake view, view room, lake view room, shuttle whistler village, lake view suite, view suite, shuttle whistler, drive whistler, bedroom lake

Topic  3
hot tub, whistler village, bedroom suite, outdoor hot tub, outdoor hot, pool hot tub, pool hot, day ski, steam room, locker room


### Using TF-IDF Spacy

In [70]:
lsa2 = TruncatedSVD(n_topics)
doc_topic2 = lsa2.fit_transform(tf_idf_spacy)
lsa2.explained_variance_ratio_

array([0.00797397, 0.01566162, 0.01383863, 0.01274385])

In [71]:
topic_word2 = pd.DataFrame(lsa2.components_.round(3),
             columns = tf_idf_spacy.columns)
topic_word2

,10 minute,20 night,30 minute,amaze view,aura restaurant,away hustle,away hustle bustle,away village,away whistler,balcony overlook,...,whistler blackcomb,whistler creekside,whistler mountain,whistler stay,whistler time,whistler village,wife stay,wish stay,wish stay long,wonderful stay
0,0.049,0.018,0.022,0.020,0.063,0.031,0.026,0.031,0.027,0.028,...,0.022,0.032,0.020,0.028,0.023,0.351,0.019,0.025,0.018,0.023
1,0.003,-0.003,-0.009,-0.011,0.005,-0.001,-0.005,-0.007,-0.005,-0.022,...,-0.015,-0.015,-0.015,0.001,0.002,-0.279,-0.005,-0.002,-0.005,-0.009
2,-0.031,-0.009,-0.009,-0.009,-0.014,-0.002,0.001,-0.016,-0.021,-0.006,...,-0.016,-0.006,-0.011,-0.025,-0.015,-0.139,0.000,0.000,0.002,0.001
3,-0.029,0.014,-0.006,0.021,0.055,-0.020,-0.014,0.022,-0.043,0.004,...,-0.010,-0.002,-0.006,0.015,0.003,-0.534,0.008,0.004,0.002,-0.001


In [72]:
display_topics(lsa2, tf_idf_spacy.columns, n_words)


Topic  0
whistler village, lake view, view lake, free shuttle, staff friendly, hot tub, shuttle service, view room, highly recommend, stay night

Topic  1
lake view, view room, lake view room, lake view suite, view suite, room lake, room lake view, stay lake, bedroom lake view, suite lake view

Topic  2
friendly helpful, staff friendly, staff friendly helpful, highly recommend, helpful staff, friendly room, stay staff, room clean, lake staff, gas fireplace

Topic  3
hot tub, view lake, highly recommend, outdoor hot, outdoor hot tub, stay night, bedroom suite, pool hot, pool hot tub, soaker tub


## Non-Negative Matrix Factorization (NMF)

### NMF CountVec

In [73]:
nmf_model3 = NMF(n_topics)
doc_topic3 = nmf_model3.fit_transform(count_vec_spacy)

In [74]:
topic_word3 = pd.DataFrame(nmf_model3.components_.round(3),
             columns = count_vec_spacy.columns)
topic_word3

,10 minute,20 night,30 minute,amaze view,aura restaurant,away hustle,away hustle bustle,away village,away whistler,balcony overlook,...,whistler blackcomb,whistler creekside,whistler mountain,whistler stay,whistler time,whistler village,wife stay,wish stay,wish stay long,wonderful stay
0,0.208,0.000,0.168,0.000,0.039,0.116,0.099,0.000,0.199,0.046,...,0.150,0.141,0.107,0.046,0.040,5.554,0.027,0.013,0.004,0.038
1,0.070,0.029,0.015,0.000,0.088,0.060,0.027,0.032,0.023,0.000,...,0.000,0.017,0.000,0.050,0.036,0.000,0.010,0.031,0.021,0.000
2,0.063,0.054,0.025,0.079,0.216,0.088,0.090,0.098,0.000,0.179,...,0.000,0.060,0.009,0.061,0.034,0.000,0.071,0.105,0.091,0.074
3,0.100,0.055,0.024,0.036,0.235,0.044,0.053,0.078,0.004,0.014,...,0.093,0.010,0.061,0.050,0.026,0.000,0.009,0.001,0.002,0.030


In [75]:
display_topics(nmf_model3, count_vec_spacy.columns, n_words)


Topic  0
whistler village, shuttle whistler, shuttle whistler village, stay whistler, drive whistler, drive whistler village, bustle whistler, minute drive, bustle whistler village, complimentary shuttle

Topic  1
lake view, view room, lake view room, view suite, lake view suite, room lake, room lake view, bedroom lake view, bedroom lake, suite lake view

Topic  2
free shuttle, view lake, shuttle service, staff friendly, friendly helpful, overlook lake, highly recommend, staff friendly helpful, stay night, free shuttle service

Topic  3
hot tub, bedroom suite, outdoor hot tub, outdoor hot, heat floor, pool hot, pool hot tub, live room, stay night, steam room


In [76]:
H3 = pd.DataFrame(doc_topic3.round(5))
H3.head()

,0,1,2,3
0,0.01171,0.01436,0.08996,0.02166
1,0.16902,0.00000,0.09721,0.00000
2,0.00176,0.01060,0.01385,0.02088
3,0.00526,0.00099,0.20057,0.02677
4,0.00622,0.00907,0.04304,0.00249


### NMF TF-IDF

In [77]:
nmf_model4 = NMF(n_topics)
doc_topic4 = nmf_model4.fit_transform(tf_idf_spacy)

In [78]:
topic_word4 = pd.DataFrame(nmf_model4.components_.round(3),
             columns = tf_idf_spacy.columns)
topic_word4

,10 minute,20 night,30 minute,amaze view,aura restaurant,away hustle,away hustle bustle,away village,away whistler,balcony overlook,...,whistler blackcomb,whistler creekside,whistler mountain,whistler stay,whistler time,whistler village,wife stay,wish stay,wish stay long,wonderful stay
0,0.161,0.054,0.078,0.080,0.155,0.094,0.081,0.085,0.106,0.115,...,0.077,0.107,0.075,0.081,0.065,1.6,0.054,0.077,0.059,0.076
1,0.067,0.007,0.008,0.000,0.080,0.036,0.018,0.023,0.030,0.000,...,0.000,0.010,0.000,0.038,0.039,0.0,0.007,0.022,0.009,0.007
2,0.000,0.000,0.000,0.000,0.007,0.016,0.021,0.003,0.000,0.010,...,0.000,0.020,0.000,0.000,0.000,0.0,0.017,0.019,0.020,0.021
3,0.003,0.024,0.009,0.014,0.065,0.000,0.003,0.045,0.000,0.000,...,0.039,0.012,0.029,0.023,0.003,0.0,0.014,0.000,0.000,0.004


In [79]:
display_topics(nmf_model4, tf_idf_spacy.columns, n_words)


Topic  0
whistler village, free shuttle, view lake, shuttle service, place stay, stay whistler, overlook lake, highly recommend, stay night, shuttle whistler

Topic  1
lake view, view room, lake view room, view suite, lake view suite, room lake, stay lake, room lake view, room spacious, stay night

Topic  2
staff friendly, friendly helpful, staff friendly helpful, highly recommend, helpful staff, room clean, friendly room, shuttle driver, stay staff, gas fireplace

Topic  3
hot tub, outdoor hot, outdoor hot tub, bedroom suite, pool hot, pool hot tub, steam room, day ski, soaker tub, room spacious


In [80]:
H4 = pd.DataFrame(doc_topic4.round(5))
H4.head()

,0,1,2,3
0,0.04998,0.01165,0.00000,0.02041
1,0.10709,0.00000,0.00000,0.00000
2,0.01613,0.00952,0.00277,0.02152
3,0.06477,0.00595,0.06904,0.00833
4,0.02791,0.00916,0.02262,0.00628


In [94]:
H4.shape

(2145, 4)

## Latent Dirichlet Allocation

### With Count Vec Spacy

In [81]:
# Convert sparse matrix of counts to a gensim corpus
corpus5 = matutils.Sparse2Corpus(X_cvspacy)

# Create lda model (equivalent to "fit" in sklearn)
lda5 = models.LdaModel( corpus=corpus5, num_topics=n_topics, 
                       id2word=id2word_cvspacy, passes=5)

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  s

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  s

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  s

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  s

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  s

In [82]:
lda5.print_topics()

[(0,
  '0.054*"lake view" + 0.037*"flat screen" + 0.034*"whistler village" + 0.026*"screen tv" + 0.023*"view room" + 0.020*"shuttle service" + 0.018*"staff friendly" + 0.017*"bedroom suite" + 0.016*"live room" + 0.016*"live area"'),
 (1,
  '0.050*"whistler village" + 0.030*"friendly helpful" + 0.029*"staff friendly" + 0.024*"highly recommend" + 0.021*"view lake" + 0.021*"shuttle service" + 0.021*"overlook lake" + 0.018*"staff friendly helpful" + 0.018*"stay long" + 0.016*"lake view"'),
 (2,
  '0.061*"whistler village" + 0.036*"stay whistler" + 0.027*"creekside gondola" + 0.025*"room spacious" + 0.025*"hot tub" + 0.020*"free shuttle" + 0.019*"enjoy stay" + 0.018*"minute walk" + 0.018*"place stay" + 0.016*"bathroom floor"'),
 (3,
  '0.041*"hot tub" + 0.034*"overlook lake" + 0.028*"mountain view" + 0.027*"main village" + 0.020*"view room" + 0.020*"view lake" + 0.019*"balcony overlook" + 0.018*"room large" + 0.018*"lake mountain" + 0.018*"balcony overlook lake"')]

In [83]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus5 = lda5[corpus5]
lda_corpus5

In [84]:
lda_docs5 = [doc for doc in lda_corpus5]

In [85]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs5[0:5]

[[(0, 0.02564294), (1, 0.02559878), (2, 0.20032778), (3, 0.74843055)],
 [(0, 0.49014235), (1, 0.032487582), (2, 0.44518298), (3, 0.032187063)],
 [(0, 0.042680513), (1, 0.57282305), (2, 0.33936968), (3, 0.04512674)],
 [(0, 0.027516076), (1, 0.34334868), (2, 0.39893478), (3, 0.23020045)],
 [(0, 0.37529382), (1, 0.48929206), (2, 0.063752614), (3, 0.07166153)]]

### Visualize LDA

In [86]:
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda5, corpus5, dictionary=lda5.id2word)
# vis

In [87]:
# # Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis


# pyLDAvis.enable_notebook()
# dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
# dash

### With TF-IDF Spacy

In [88]:
# Convert sparse matrix of counts to a gensim corpus
corpus6 = matutils.Sparse2Corpus(X_tfispacy)

# Create lda model (equivalent to "fit" in sklearn)
lda6 = models.LdaModel( corpus=corpus6, num_topics=n_topics, 
                       id2word=id2word_tfispacy, passes=5)

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  s

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  s

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  s

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  s

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  s

In [89]:
lda6.print_topics()

[(0,
  '0.051*"stay whistler" + 0.036*"room spacious" + 0.023*"place stay" + 0.020*"whistler mountain" + 0.019*"place stay whistler" + 0.017*"whistler village" + 0.017*"comfy bed" + 0.017*"whistler time" + 0.016*"look forward" + 0.016*"live area"'),
 (1,
  '0.035*"whistler village" + 0.027*"overlook lake" + 0.021*"highly recommend" + 0.017*"food restaurant" + 0.016*"minute walk" + 0.015*"view lake" + 0.015*"room bathroom" + 0.014*"stay whistler" + 0.014*"shuttle service" + 0.014*"shuttle driver"'),
 (2,
  '0.029*"staff friendly" + 0.027*"friendly helpful" + 0.024*"whistler village" + 0.021*"flat screen" + 0.019*"staff friendly helpful" + 0.016*"soaker tub" + 0.015*"huge bathroom" + 0.015*"stay lake" + 0.015*"pet friendly" + 0.014*"bedroom suite"'),
 (3,
  '0.030*"hot tub" + 0.024*"view room" + 0.022*"lake view" + 0.017*"locker room" + 0.015*"whistler village" + 0.015*"lake view room" + 0.014*"free shuttle" + 0.013*"heat floor" + 0.012*"studio room" + 0.011*"king size"')]

In [90]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus6 = lda6[corpus6]
lda_corpus6

In [91]:
lda_docs6 = [doc for doc in lda_corpus6]

In [92]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs6[0:5]

[[(0, 0.61357903), (1, 0.21352485), (2, 0.06371603), (3, 0.1091801)],
 [(0, 0.091645144), (1, 0.495682), (2, 0.077739745), (3, 0.33493313)],
 [(0, 0.09813918), (1, 0.4764337), (2, 0.34070477), (3, 0.08472235)],
 [(0, 0.19844057), (1, 0.07369504), (2, 0.093123324), (3, 0.63474107)],
 [(0, 0.09484003), (1, 0.100205176), (2, 0.70811146), (3, 0.09684336)]]

## Regression to predict helpfulness

In [96]:
X = H4
X.head()

,0,1,2,3
0,0.04998,0.01165,0.00000,0.02041
1,0.10709,0.00000,0.00000,0.00000
2,0.01613,0.00952,0.00277,0.02152
3,0.06477,0.00595,0.06904,0.00833
4,0.02791,0.00916,0.02262,0.00628


In [98]:
y.head()

,helpful_vote
0,23
1,38
2,4
3,27
4,1


In [100]:
# Create an empty model
lr = LinearRegression()

# Get the train and test data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)


In [126]:
#Pipeline
degree = 3
est = make_pipeline(PolynomialFeatures(degree), lr)

In [127]:
scoring = ['neg_mean_squared_error', 'r2']

scores = cross_validate(est, X_train, y_train,
                              scoring = scoring, cv = 7,
                              n_jobs = 3,
                              return_train_score = True)

In [128]:
cv_results = pd.DataFrame(scores)
cv_results

,fit_time,score_time,test_neg_mean_squared_error,train_neg_mean_squared_error,test_r2,train_r2
0,0.013649,0.004446,-2894.414430,-2665.316769,-0.042940,0.041890
1,0.010942,0.004533,-3662.244898,-4110.077498,-0.651355,-0.428146
2,0.008187,0.004076,-1445.239026,-2914.204466,-0.001260,0.031172
3,0.005834,0.004217,-2630.759184,-3036.333107,-0.201275,-0.053096
4,0.007889,0.004410,-5189.302041,-2525.299116,-0.118606,-0.019927
5,0.008175,0.004341,-1709.155102,-3204.469748,-0.254036,-0.060727
6,0.006906,0.003781,-5318.897959,-2613.403127,-0.108997,-0.067562


## Random Forest Regressor

In [122]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_predict

In [123]:
def rfr_model(X, y):
# Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,                         n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],                               random_state=False, verbose=False)
# Perform K-Fold CV
    scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')

    return scores

In [124]:
rfr = rfr_model(X, np.ravel(y))

In [ ]:
predictions = cross_val_predict(rfr, X, y, cv=10)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


# Split the data into training/testing sets
diabetes_X_train = X[:-20]
diabetes_X_test = X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = y[:-20]
diabetes_y_test = y[-20:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(diabetes_y_test, diabetes_y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(diabetes_y_test, diabetes_y_pred))

# Plot outputs
plt.scatter(diabetes_X_test, diabetes_y_test,  color='black')
plt.plot(diabetes_X_test, diabetes_y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [130]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [131]:
regr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)

In [135]:
regr.fit(X_train, np.ravel(y_train))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [136]:
print(regr.feature_importances_)

[0.22573733 0.51294773 0.15420963 0.10710531]
